In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=c958fbab1a764cdf54af43c90abc6765be200c49015e5ec67b856927c6d9dac8
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!git clone https://github.com/clab/fast_align

Cloning into 'fast_align'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 213 (delta 2), reused 4 (delta 2), pack-reused 204
Receiving objects: 100% (213/213), 70.68 KiB | 3.53 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [ ]:
!mkdir fast_align/build
!cd fast_align/build && cmake .. && make

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Could NOT find SparseHash (missing: SPARSEHASH_INCLUDE_DIR) 
-- Configuring done
-- Generating done
-- Build files have been written to: /content/fast_align/build
[ 16%] Building CXX o

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence


from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu

import numpy as np
import pandas as pd

from scipy.optimize import linear_sum_assignment

import sys
import os
import random
import math
import codecs

from collections import defaultdict
from tqdm.notebook import tqdm

import wandb

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


# Translation

In [ ]:
!tar -xvf '/content/drive/MyDrive/fr-en.tgz' -C '.'

europarl-v7.fr-en.en
europarl-v7.fr-en.fr


In [ ]:
!cp '/content/drive/MyDrive/train.en.txt' '.'

In [ ]:
!cp '/content/drive/MyDrive/train.de.txt' '.'

^C


In [ ]:
!cp '/content/drive/MyDrive/dict.en-de.txt' '.'

In [ ]:
!cp '/content/drive/MyDrive/newstest2014.en.txt' '.'

In [ ]:
!cp '/content/drive/MyDrive/newstest2014.de.txt' '.'

In [ ]:
with open('/content/europarl-v7.fr-en.en', 'r') as f:
    print(f.readlines()[:2])
with open('/content/europarl-v7.fr-en.fr', 'r') as f:
    print(f.readlines()[:2])

with open('/content/train.de.txt', 'r') as f:
    print(f.readlines()[:2])
with open('/content/train.en.txt', 'r') as f:
    print(f.readlines()[:2])
with open('/content/dict.en-de.txt', 'r') as f:
    print(''.join(f.readlines()[:100]))
with open('/content/newstest2014.en.txt', 'r') as f:
    t = f.readlines()
    print(len(t))
    print(t[:2])
with open('/content/newstest2014.de.txt', 'r') as f:
    t = f.readlines()
    print(len(t))
    print(t[:2])

In [ ]:
with open('/content/europarl-v7.fr-en.en', 'r') as f:
    print('\n'.join(f.readlines()[:10]))

Resumption of the session

I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.

Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.

You have requested a debate on this subject in the course of the next few days, during this part-session.

In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.

Please rise, then, for this minute' s silence.

(The House rose and observed a minute' s silence)

Madam President, on a point of order.

You will be aware from the press and television that there have been a number of bomb explos

In [ ]:
def read_data(read_paths, save_path, maxlen=30, langs=['german', 'english'], maxsize=5000000):
    lang_from, lang_to = langs

    input_file_lang1 = codecs.open(read_paths[0], 'r', 'utf-8')
    input_file_lang2 = codecs.open(read_paths[1], 'r', 'utf-8')
    output_file = codecs.open(save_path, 'w', 'utf-8')

    for i, (line1, line2) in tqdm(enumerate(zip(input_file_lang1, input_file_lang2))):
        content1, content2 = line1.strip().lower(), line2.strip().lower()
        tokenized_content1 = word_tokenize(content1, language=langs[0])
        tokenized_content2 = word_tokenize(content2, language=langs[1])
        if len(tokenized_content1) < maxlen \
            and len(tokenized_content2) < maxlen \
            and len(tokenized_content1) > 1 \
            and len(tokenized_content2) > 1:
            output_file.write(content1 + ' ||| ' + content2 + '\n')

        if i > maxsize:
            break

    input_file_lang1.close()
    input_file_lang2.close()
    output_file.close()


def read_vocab_from_corpus(vocab_path, save_path, capacity=50000, lang='english'):
    counts = defaultdict(int)
    input_file = codecs.open(vocab_path, 'r', 'utf-8')
    output_file = codecs.open(save_path, 'w', 'utf-8')

    for line in tqdm(input_file):
        content = line.strip().lower()
        tokenized_content = word_tokenize(content, language=lang)
        for word in tokenized_content:
            counts[word] += 1

    picked_words = sorted(counts.keys(), key=lambda x: counts[x], reverse=True)[:capacity]
    for word in picked_words:
        output_file.write(word + '\n')
            
    input_file.close()
    output_file.close()

In [ ]:
!rm -r vocab
!mkdir vocab
read_vocab_from_corpus('/content/train.de.txt', '/content/vocab/wmt14.de', lang='german')
read_vocab_from_corpus('/content/train.en.txt', '/content/vocab/wmt14.en', lang='english')
read_data(['/content/newstest2014.de.txt', '/content/newstest2014.en.txt'], '/content/parallel_en_de_newstest.txt')
read_data(['/content/train.de.txt', '/content/train.en.txt'], '/content/parallel_en_de_train.txt')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
with open('/content/parallel_en_de_newstest.txt', 'r') as f:
    t = f.readlines()
    print(len(t))
    print('\n'.join(t[:10]))

1639
orlando bloom und miranda kerr lieben sich noch immer ||| orlando bloom and miranda kerr still love each other

schauspieler orlando bloom und model miranda kerr wollen künftig getrennte wege gehen . ||| actors orlando bloom and model miranda kerr want to go their separate ways .

in einem interview sagte bloom jedoch , dass er und kerr sich noch immer lieben . ||| however , in an interview , bloom has said that he and kerr still love each other .

schauspieler orlando bloom hat sich zur trennung von seiner frau , topmodel miranda kerr , geäußert . ||| actor orlando bloom announced his separation from his wife , supermodel miranda kerr .

&quot; wir werden uns gegenseitig unterstützen und lieben als eltern von flynn &quot; . ||| &quot; we &apos;re going to support one another and love each other as parents to flynn &quot; .

kerr und bloom sind seit 2010 verheiratet , im jahr 2011 wurde ihr söhnchen flynn geboren . ||| kerr and bloom have been married since 2010 . their son flynn 

In [ ]:
!./fast_align/build/fast_align -i parallel_en_de_train.txt -d -o -v > forward.align.train
!./fast_align/build/fast_align -i parallel_en_de_train.txt -d -o -v -r > reverse.align.train
!./fast_align/build/atools -i forward.align.train -j reverse.align.train -c grow-diag-final-and > symmetric_train_align.txt

!./fast_align/build/fast_align -i parallel_en_de_newstest.txt -d -o -v > forward.align.newstest
!./fast_align/build/fast_align -i parallel_en_de_newstest.txt -d -o -v -r > reverse.align.newstest
!./fast_align/build/atools -i forward.align.newstest -j reverse.align.newstest -c grow-diag-final-and > symmetric_newstest_align.txt

In [ ]:
with open('/content/symmetric_train_align.txt', 'r') as f:
    print(len(f.readlines()))
with open('/content/symmetric_newstest_align.txt', 'r') as f:
    print(len(f.readlines()))

1781990
1639


In [5]:
# !cp '/content/symmetric_train_align.txt' '/content/drive/MyDrive/CTC_in_NLP/symmetric_train_align.txt'
# !cp '/content/symmetric_newstest_align.txt' '/content/drive/MyDrive/CTC_in_NLP/symmetric_newstest_align.txt'
# !cp '/content/forward.align.newstest' '/content/drive/MyDrive/CTC_in_NLP/forward.align.newstest'
# !cp '/content/forward.align.train' '/content/drive/MyDrive/CTC_in_NLP/forward.align.train'
# !cp '/content/reverse.align.newstest' '/content/drive/MyDrive/CTC_in_NLP/reverse.align.newstest'
# !cp '/content/reverse.align.train' '/content/drive/MyDrive/CTC_in_NLP/reverse.align.train'
!cp '/content/drive/MyDrive/CTC_in_NLP/symmetric_train_align.txt' '/content/symmetric_train_align.txt'
!cp '/content/drive/MyDrive/CTC_in_NLP/symmetric_newstest_align.txt' '/content/symmetric_newstest_align.txt'
!cp '/content/drive/MyDrive/CTC_in_NLP/forward.align.newstest' '/content/forward.align.newstest'
!cp '/content/drive/MyDrive/CTC_in_NLP/forward.align.train' '/content/forward.align.train'
!cp '/content/drive/MyDrive/CTC_in_NLP/reverse.align.newstest' '/content/reverse.align.newstest'
!cp '/content/drive/MyDrive/CTC_in_NLP/reverse.align.train' '/content/reverse.align.train'

In [6]:
# !cp '/content/parallel_en_de_newstest.txt' '/content/drive/MyDrive/CTC_in_NLP/parallel_en_de_newstest.txt'
# !cp '/content/parallel_en_de_train.txt' '/content/drive/MyDrive/CTC_in_NLP/parallel_en_de_train.txt'
# !cp '/content/vocab/wmt14.de' '/content/drive/MyDrive/CTC_in_NLP/wmt14.de'
# !cp '/content/vocab/wmt14.en' '/content/drive/MyDrive/CTC_in_NLP/wmt14.en'
!rm -r vocab
!mkdir vocab
!cp '/content/drive/MyDrive/CTC_in_NLP/parallel_en_de_newstest.txt' '/content/parallel_en_de_newstest.txt' 
!cp '/content/drive/MyDrive/CTC_in_NLP/parallel_en_de_train.txt' '/content/parallel_en_de_train.txt' 
!cp '/content/drive/MyDrive/CTC_in_NLP/wmt14.de' '/content/vocab/wmt14.de' 
!cp '/content/drive/MyDrive/CTC_in_NLP/wmt14.en' '/content/vocab/wmt14.en' 

rm: cannot remove 'vocab': No such file or directory


In [7]:
class Vocab:
    def __init__(self, lang='english', capacity=20000, maxsize=10000000):
        self.t2i = {'<PAD>': 0, '<UNK>': 1, '<CTC>': 2, '<SOS>': 3, '<EOS>': 4}
        self.i2t = {0: '<PAD>', 1: '<UNK>', 2: '<CTC>', 3: '<SOS>', 4: '<EOS>'}
        self.lang = lang
        self.capacity = capacity
        self.maxsize = maxsize

    def load(self, path):
        with open(path, 'r') as f:
            for line in tqdm(f.readlines()):
                word = line.strip().lower()
                self.t2i[word] = len(self.t2i)
                self.i2t[self.t2i[word]] = word
        return self
                
class TrDataset(Dataset):
    def __init__(self, paths, vocabs, langs):
        self.vocabs = dict([(lang, vocab) for lang, vocab in zip(langs, vocabs)])
        self.path, self.alignment_path = paths
        self.data = dict([(lang, []) for lang in langs])
        self.alignments = []
        self.langs = langs
        self.delim = ' ||| '
        
        input_file = codecs.open(self.path, 'r', 'utf-8')
        alignment_file = codecs.open(self.alignment_path, 'r', 'utf-8')

        for input_line, alignment_line in tqdm(zip(input_file, alignment_file)):
            lines = input_line.strip().split(self.delim)
            unk_count = dict(zip(langs, [0 for lang in langs]))
            lengths = dict()
            for line, lang in zip(lines, langs):
                content = line.strip().lower()
                tokenized_content = word_tokenize(content, language=lang)
                content_tokens = [self.vocabs[lang].t2i['<SOS>']]
                for word in tokenized_content:
                    if word in self.vocabs[lang].t2i:
                        content_tokens.append(self.vocabs[lang].t2i[word])
                    else:
                        unk_count[lang] += 1
                        content_tokens.append(self.vocabs[lang].t2i['<UNK>'])
                content_tokens.append(self.vocabs[lang].t2i['<EOS>'])
                self.data[lang].append(content_tokens)
                lengths[lang] = len(content_tokens)
            if unk_count[langs[0]] > 0.4 * lengths[langs[0]] or unk_count[langs[1]] > 0.4 * lengths[langs[1]]:
                self.data[langs[0]].pop()
                self.data[langs[1]].pop()
                continue
            alignment_matrix = torch.zeros((lengths[langs[1]], lengths[langs[0]]), dtype=torch.int64)
            for alignment in alignment_line.strip().split(' '):
                i, j = map(int, alignment.split('-'))
                alignment_matrix[j + 1][i + 1] += 1
            alignment_matrix[0, 0] = 1
            alignment_matrix[-1, -1] = 1

            self.alignments.append(alignment_matrix)
                    
        
    def __getitem__(self, idx):
        return [torch.LongTensor(self.data[lang][idx]) for lang in self.langs] + [torch.LongTensor(self.alignments[idx])]

    def __len__(self):
        return len(self.data[self.langs[0]])

def collate_fn(batch):
    l1, l2, m = list(zip(*batch))
    l1, l2 = pad_sequence(l1 + l2, batch_first=True)[:len(l1)], pad_sequence(l2 + l1, batch_first=True)[:len(l2)]
    m = list(m)
    for i, matrix in enumerate(m):
        eye = torch.eye(max(l1.shape[1], l2.shape[1]))
        eye[:matrix.shape[0], :matrix.shape[1]] = matrix
        m[i] = eye
    m = torch.stack(m, dim=0)
    return l1, l2, m

In [8]:
import gc
gc.collect()

0

In [9]:
vocabs = [Vocab(lang='german').load('/content/vocab/wmt14.de'),
          Vocab(lang='english').load('/content/vocab/wmt14.en')]
langs = ['german', 'english']
train_dataset = TrDataset(['/content/parallel_en_de_train.txt', '/content/symmetric_train_align.txt'], vocabs, langs)
test_dataset = TrDataset(['/content/parallel_en_de_newstest.txt', '/content/symmetric_newstest_align.txt'], vocabs, langs)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [10]:
len(train_dataset), len(test_dataset)

(1774415, 1638)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [12]:
for batch in val_loader:
    print(batch[0].shape)
    break

torch.Size([64, 31])


In [27]:
class Encoder(nn.Module):
    def __init__(self, index_dim, embedding_dim=256, hidden_dim=256, dropout=0.2, layers=2):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(index_dim, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=layers, dropout=dropout, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.directions = 2
        self.layers = layers

    def forward(self, x):
        mask = (x != 0).to(torch.long)
        lengths = mask.sum(dim=1).to('cpu')

        emb = self.embedding(x)
        emb = pack_padded_sequence(emb, lengths, batch_first=True, enforce_sorted=False)
        out, hidden = self.lstm(emb)
        out, _ = pad_packed_sequence(out, batch_first=True)
        return self.dropout(out), hidden


class LSTMAttention(nn.Module):
    def __init__(self, hidden_dim, attention_dim, directions=2):
        super(LSTMAttention, self).__init__()
        self.W1 = nn.Linear(hidden_dim * directions, attention_dim)
        self.W2 = nn.Linear(hidden_dim * directions, attention_dim)
        self.V = nn.Linear(attention_dim, 1)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)    

    def forward(self, hidden, enc_out):
        score = self.V(self.tanh(self.W1(enc_out) + self.W2(hidden.view(hidden.shape[1], -1).unsqueeze(1)))).squeeze(-1)
        attention_weights = self.softmax(score)
        assert len(attention_weights.shape) == 2
        
        context_vector = attention_weights.unsqueeze(-1) * enc_out
        context_vector = context_vector.sum(axis=1)
        return context_vector


class Decoder(nn.Module):
    def __init__(self, index_dim, embedding_dim=256, hidden_dim=256, attention_dim=128, enc_directions=2):
        super(Decoder, self).__init__()
        self.attention = LSTMAttention(hidden_dim, attention_dim)
        self.embedding = nn.Embedding(index_dim, embedding_dim)
        self.linear = nn.Linear(hidden_dim * enc_directions, index_dim)
        self.lstm = nn.LSTM(embedding_dim + hidden_dim * enc_directions, hidden_dim * enc_directions, batch_first=True)
        self.relu = nn.ReLU()

    def forward(self, x, hidden, enc_out):
        context_vector = self.attention(hidden[0], enc_out)
        emb = self.relu(self.embedding(x))
        emb = torch.cat([context_vector.unsqueeze(1), emb], dim=-1)
        out, hidden = self.lstm(emb, hidden)
        out = self.linear(out)
        return out, hidden


class Attention(nn.Module):
    def __init__(self, hidden_dim, attention_dim):
        super(Attention, self).__init__()
        self.Q = nn.Linear(hidden_dim, attention_dim)
        self.K = nn.Linear(hidden_dim, attention_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.softmax = nn.Softmax(dim=1)    

    def forward(self, x):
        Qw = self.Q(x)
        Kw = self.K(x)
        Vw = self.V(x)
        attention_weights = (self.softmax(Qw@(Kw.permute(0, 2, 1)))@Vw).squeeze(-1)
        assert len(attention_weights.shape) == 2
        
        context_vector = attention_weights.unsqueeze(-1) * x
        return context_vector

class GatedAttention(nn.Module):
    def __init__(self, hidden_dim, attention_dim):
        super(GatedAttention, self).__init__()
        self.Q = nn.Linear(hidden_dim, attention_dim)
        self.K = nn.Linear(hidden_dim, attention_dim)
        self.V = nn.Linear(attention_dim, 1)
        self.softmax = nn.Softmax(dim=1)    
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        Qw = self.Q(x)
        Kw = self.K(x)

        g = self.sigmoid(self.V(Qw)).squeeze(-1)
        M = torch.diag(torch.ones((x.shape[1])) * -torch.inf).repeat(x.shape[0], 1, 1).to(device=x.device)
        D = torch.diag_embed(g).to(device=x.device)
        I = torch.diag(torch.ones((x.shape[1]))).repeat(x.shape[0], 1, 1).to(device=x.device)
        P = self.softmax(M + Qw@(Kw.permute(0, 2, 1)))
        P = D + (I - D) @ P
        return P

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout = 0.1, max_len = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)


# The duplication predictor and grouping predictor modules consist of 
# a convolutional layer, 
# ReLU activation, 
# layer normalization, 
# dropout,
# a projection layer (convs + linear -> how????), same as the phoneme duration predictor in FastSpeech (Ren et al., 2019), which is a parallel text-to-speech model.

class GroupsLoss(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, probs):
        return -torch.log(self.softmax(probs)).mean(dim=-1)


class LengthClassDuplicationModule(nn.Module):
    def __init__(self, input_dim, hidden_dim, max_duplication=3):
        super().__init__()
        self.conv = nn.Conv1d(input_dim, hidden_dim, 3, padding=1)
        self.relu = nn.ReLU()
        self.projection = nn.Linear(hidden_dim, max_duplication)
        self.ln = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.max_duplication = max_duplication

    def forward(self, x):
        lengths_probs = self.projection(self.dropout(self.ln(self.relu(self.conv(x.permute(0, 2, 1)).permute(0, 2, 1)))))
        lengths = torch.cumsum(torch.argmax(lengths_probs, dim=-1).unsqueeze(1) + 1, dim=-1)
        duplication_matrix = torch.zeros((x.shape[0], x.shape[1] * self.max_duplication, x.shape[1])) + torch.arange(1, 1 + x.shape[1] * self.max_duplication).unsqueeze(0).unsqueeze(2)
        duplication_matrix = duplication_matrix.to(device=x.device)
        duplication_matrix[:, :, 0] = torch.clamp(lengths[:, :, 0] - duplication_matrix[:, :, 0] + 1, 0, 1)
        duplication_matrix[:, :, 1:] = torch.clamp(lengths[:, :, 1:] - duplication_matrix[:, :, 1:] + 1, 0, 1) * torch.clamp(duplication_matrix[:, :, 1:] - lengths[:, :, :-1], 0, 1)

        #nn.parallel.parallel_apply([lambda x: torch.repeat_interleave(x[i], lengths[i], dim=1) for i in range(x.shape[0])], x)
        return lengths_probs, duplication_matrix


class LengthRegDuplicationModule(nn.Module):
    def __init__(self, input_dim, hidden_dim, max_duplication=3):
        super().__init__()
        self.conv = nn.Conv1d(input_dim, hidden_dim, 3, padding=1)
        self.relu = nn.ReLU()
        self.projection = nn.Linear(hidden_dim, 1)
        self.ln = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.max_duplication = max_duplication

    def forward(self, x):
        lengths = torch.clamp(self.projection(self.dropout(self.ln(self.relu(self.conv(x.permute(0, 2, 1)).permute(0, 2, 1))))).squeeze(-1), 0, self.max_duplication - 1)
        lengths = torch.cumsum(lengths.floor().unsqueeze(1) + 1, dim=-1)
        duplication_matrix = torch.zeros((x.shape[0], x.shape[1] * self.max_duplication, x.shape[1])) + torch.arange(1, 1 + x.shape[1] * self.max_duplication).unsqueeze(0).unsqueeze(2)
        duplication_matrix = duplication_matrix.to(device=x.device)
        duplication_matrix[:, :, 0] = torch.clamp(lengths[:, :, 0] - duplication_matrix[:, :, 0] + 1, 0, 1)
        duplication_matrix[:, :, 1:] = torch.clamp(lengths[:, :, 1:] - duplication_matrix[:, :, 1:] + 1, 0, 1) * torch.clamp(duplication_matrix[:, :, 1:] - lengths[:, :, :-1], 0, 1)

        #nn.parallel.parallel_apply([lambda x: torch.repeat_interleave(x[i], lengths[i], dim=1) for i in range(x.shape[0])], x)
        return duplication_matrix


class GroupingModule(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.conv = nn.Conv1d(input_dim, hidden_dim, 3, padding=1)
        self.relu = nn.ReLU()
        self.projection = nn.Linear(hidden_dim, 2)
        self.ln = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        groups_probs = self.projection(self.dropout(self.ln(self.relu(self.conv(x.permute(0, 2, 1)).permute(0, 2, 1)))))
        groups = torch.argmax(groups_probs, dim=-1) # [batch, length]
        groups[:, 0] = 0
        start_groups = groups.nonzero(as_tuple=True)
        last_group = groups.shape[1]
        groups = torch.cumsum(torch.ones(groups.shape).to(device=x.device), dim=-1)
        groups[:, 0] = 0
        groups[start_groups[0], start_groups[1]] = 0
        groups[:, -1] = last_group
        groups = torch.gather(groups, 1, groups.ne(0).to(dtype=torch.int).argsort(dim=1, descending=True, stable=True)).unsqueeze(2)
        grouping_matrix = torch.zeros((x.shape[0], x.shape[1], x.shape[1])) + torch.arange(1, 1 + x.shape[1]).unsqueeze(0).unsqueeze(1)
        grouping_matrix = grouping_matrix.to(device=x.device)
        grouping_matrix[:, 0, :] = torch.clamp(groups[:, 0, :] - grouping_matrix[:, 0, :] + 1, 0, 1)
        grouping_matrix[:, 1:, :] = torch.clamp(groups[:, 1:, :] - grouping_matrix[:, 1:, :] + 1, 0, 1) * torch.clamp(grouping_matrix[:, 1:, :] - groups[:, :-1, :], 0, 1)

        #nn.parallel.parallel_apply([lambda x: torch.repeat_interleave(x[i], lengths[i], dim=1) for i in range(x.shape[0])], x)
        return groups_probs, grouping_matrix


# The permutation predictor in Aligner consists of three encoder layers: 
# pre-network, query/key network, and single-head attention module for the outputs.
class PermutationModule(nn.Module):
    def __init__(self, input_dim, hidden_dim, attention_dim):
        super().__init__()
        self.encoder = nn.ModuleList([
            nn.Linear(input_dim, hidden_dim), Attention(hidden_dim, attention_dim), 
            nn.Linear(hidden_dim, hidden_dim), Attention(hidden_dim, attention_dim)])
        self.pre_net = nn.Linear(hidden_dim, hidden_dim)
        self.gated_attention = GatedAttention(hidden_dim, attention_dim)

    def forward(self, x):
        for layer in self.encoder:
            x = layer(x)
        return self.gated_attention(self.pre_net(x))


class CTCDecoder(nn.Module):
    def __init__(self, index_dim, hidden_dim=512):
        super(CTCDecoder, self).__init__()
        self.ctc_linear = nn.Linear(hidden_dim, index_dim)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        # x = torch.repeat_interleave(x.permute(1, 0, 2), 1, dim=1)
        return self.softmax(self.ctc_linear(x.permute(1, 0, 2)))


class CTCTransformerDecoder(nn.Module):
    def __init__(self, index_dim, hidden_dim=512, heads=4):
        super(CTCTransformerDecoder, self).__init__()
        self.embedding = nn.Embedding(index_dim, hidden_dim, padding_idx=0)
        self.decoder = nn.TransformerDecoderLayer(d_model=hidden_dim, nhead=heads, dim_feedforward=512)
        self.ctc_linear = nn.Linear(hidden_dim, index_dim)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, x):
        # x = torch.repeat_interleave(x.permute(1, 0, 2), 1, dim=1)
        # y = torch.zeros(x.shape).uniform_(-0.1, 0.1).to(device=x.device) 
        # y = self.embedding(self.softmax(self.ctc_linear(x)).argmax(-1))
        return self.softmax(self.ctc_linear(self.decoder(x, x))).permute(1, 0, 2)


class Seq2CTC(nn.Module):
    def __init__(self, encoder_index_dim, decoder_index_dim, hidden_dim=256):
        super(Seq2CTC, self).__init__()
        self.encoder = Encoder(encoder_index_dim, hidden_dim=hidden_dim)
        self.ctc = CTCDecoder(decoder_index_dim)
        # self.output_mask = OutputMask(2 * hidden_dim)
        self.encoder_index_dim = encoder_index_dim

    def forward(self, x, y=None):
        batch_size = x.shape[0]
        device = x.device

        input_length = x.shape[1]

        enc_out, _ = self.encoder(x)

        enc_out = nn.functional.pad(enc_out, (0, 0, 0, enc_out.shape[1]), "constant", 0)
        
        # mask = self.output_mask(enc_out.permute(1, 0, 2))
        # ctc = torch.mul(mask.unsqueeze(-1), self.ctc(enc_out.permute(1, 0, 2)))

        return self.ctc(enc_out.permute(1, 0, 2))

class Transformer2CTC(nn.Module):
    def __init__(self, encoder_index_dim, decoder_index_dim, embedding_dim=256, 
                 heads=4, hidden_dim=256, layers=4, dropout = 0.3):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(embedding_dim, heads, hidden_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, layers)
        self.encoder = nn.Embedding(encoder_index_dim, embedding_dim)
        self.embedding_dim = embedding_dim
        self.decoder = CTCDecoder(decoder_index_dim, hidden_dim)
        # self.output_mask = OutputMask(hidden_dim=hidden_dim)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, mask):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        # x = nn.functional.pad(x, (0, 0, 0, x.shape[0]), "constant", 0)
        x = self.encoder(x) * math.sqrt(self.embedding_dim)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, mask)
        # out_mask = self.output_mask(output)
        # output = torch.mul(out_mask.unsqueeze(-1), self.decoder(output))
        return self.decoder(output)


class Transformer2TransformerCTC(nn.Module):
    def __init__(self, encoder_index_dim, decoder_index_dim, embedding_dim=256, 
                 heads=4, hidden_dim=256, layers=4, dropout = 0.3):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(embedding_dim, heads, hidden_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, layers)
        self.encoder = nn.Embedding(encoder_index_dim, embedding_dim)
        self.embedding_dim = embedding_dim
        self.decoder = CTCTransformerDecoder(decoder_index_dim, hidden_dim)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, mask):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        # x = nn.functional.pad(x, (0, 0, 0, x.shape[0]), "constant", 0)
        x = self.encoder(x) * math.sqrt(self.embedding_dim)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, mask)
        # out_mask = self.output_mask(output)
        # output = torch.mul(out_mask.unsqueeze(-1), self.decoder(output))
        return self.decoder(output)


class AligNART(nn.Module):
    def __init__(self, encoder_index_dim, decoder_index_dim, embedding_dim=256, 
                 heads=4, hidden_dim=256, layers=4, dropout = 0.3):
        super().__init__()
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(embedding_dim, heads, hidden_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, layers)
        self.encoder = nn.Embedding(encoder_index_dim, embedding_dim)
        self.embedding_dim = embedding_dim
        self.decoder = CTCTransformerDecoder(decoder_index_dim, hidden_dim)
        self.permute = PermutationModule(hidden_dim, hidden_dim, hidden_dim)
        self.group = GroupingModule(hidden_dim, hidden_dim)
        self.duplicate = LengthClassDuplicationModule(hidden_dim, hidden_dim)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, mask, gt=None):
        x = self.encoder(x) * math.sqrt(self.embedding_dim)
        x = self.pos_encoder(x)
        enc_output = self.transformer_encoder(x, mask).permute(1, 0, 2)
        duplication_probs, duplication_matrix = self.duplicate(enc_output)
        output = self.pos_encoder(gt[0]@enc_output) if gt is not None else self.pos_encoder(duplication_matrix@enc_output)
        permutation_matrix = self.permute(output)
        if gt is not None:
            output = gt[1]@output 
        else:
            permutation_matrix = permutation_matrix.cpu().detach().numpy()
            eye = np.eye(permutation_matrix.shape[1], permutation_matrix.shape[2])
            for i in range(permutation_matrix.shape[0]):
                try:
                    permutation_matrix[i] += 1e-16
                    xs, ys = linear_sum_assignment(-1 * np.log(permutation_matrix[i]))
                    permutation_matrix[i] *= 0
                    permutation_matrix[i, xs, ys] = 1
                    ones = int(permutation_matrix[i].sum())
                    permutation_matrix[i][ones:, ones:] = eye[ones:, ones:]
                except Exception as e:
                    permutation_matrix[i] = eye
                    print("Warning: permutation_matrix ", e)
            permutation_matrix = torch.Tensor(permutation_matrix).to(device=x.device)
            output = permutation_matrix@output
        grouping_probs, grouping_matrix = self.group(output)
        if gt is not None:
            alignment = gt[2]@gt[1]@gt[0]
            output = gt[2]@output / torch.clamp(alignment.sum(dim=2).unsqueeze(2), min=1)
        else:
            alignment = grouping_matrix@permutation_matrix@duplication_matrix
            output = grouping_matrix@output /  torch.clamp(alignment.sum(dim=2).unsqueeze(2), min=1)
        #output = gt[2]@output if gt is not None else grouping_matrix@output
        output = self.decoder(output.permute(1, 0, 2))
        return output, (duplication_probs, duplication_matrix, permutation_matrix, grouping_probs, grouping_matrix)


class CTCAligner(nn.Module):
    def __init__(self, encoder_index_dim, decoder_index_dim, embedding_dim=256, 
                 heads=4, hidden_dim=256, layers=4, dropout = 0.3):
        super().__init__()
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(embedding_dim, heads, hidden_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, layers)
        self.encoder = nn.Embedding(encoder_index_dim, embedding_dim)
        self.embedding_dim = embedding_dim
        self.decoder = CTCTransformerDecoder(decoder_index_dim, hidden_dim)
        self.permute = PermutationModule(hidden_dim, hidden_dim, hidden_dim)
        self.duplicate = LengthRegDuplicationModule(hidden_dim, hidden_dim)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, mask, gt=None):
        x = self.encoder(x) * math.sqrt(self.embedding_dim)
        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, mask).permute(1, 0, 2)
        duplication_matrix = self.duplicate(output)
        # if gt is not None:
        #     print(gt[0].shape, output.shape)
        output = gt[0]@output if gt is not None else self.pos_encoder(duplication_matrix@output)
        permutation_matrix = self.permute(output)
        if gt is not None:
            output = gt[1]@output 
        else:
            permutation_matrix = permutation_matrix.cpu().detach().numpy()
            eye = np.eye(permutation_matrix.shape[1], permutation_matrix.shape[2])
            for i in range(permutation_matrix.shape[0]):
                try:
                    permutation_matrix[i] += 1e-16
                    xs, ys = linear_sum_assignment(-1 * np.log(permutation_matrix[i]))
                    permutation_matrix[i] *= 0
                    permutation_matrix[i, xs, ys] = 1
                    ones = int(permutation_matrix[i].sum())
                    permutation_matrix[i][ones:, ones:] = eye[ones:, ones:]
                except Exception as e:
                    permutation_matrix[i] = eye
                    print("Warning: permutation_matrix ", e)
            permutation_matrix = torch.Tensor(permutation_matrix).to(device=x.device)
            output = permutation_matrix@output
        output = self.decoder(output.permute(1, 0, 2))
        return output, (duplication_matrix, permutation_matrix)


class Seq2Seq(nn.Module):
    def __init__(self, encoder_index_dim, decoder_index_dim, hidden_dim=256):
        super().__init__()
        self.encoder = Encoder(encoder_index_dim, hidden_dim=hidden_dim)
        self.decoder = Decoder(decoder_index_dim, hidden_dim=hidden_dim)

        self.encoder_index_dim = encoder_index_dim
        self.decoder_index_dim = decoder_index_dim


    def forward(self, x, y, use_teacher_forcing):
        if y is not None:
            assert x.device == y.device
            assert x.shape[0] == y.shape[0]

        batch_size = x.shape[0]
        device = x.device

        if y is not None:
            target_length = y.shape[1]
        else:
            target_length = x.shape[1]

        enc_out, hidden = self.encoder(x)
        out = x[:, 0].unsqueeze(1)

        hidden = [hidden[0].view(self.encoder.layers, batch_size, -1).mean(dim=0).unsqueeze(0),
                hidden[1].view(self.encoder.layers, batch_size, -1).mean(dim=0).unsqueeze(0)]

        outputs = torch.zeros(batch_size, target_length, self.decoder_index_dim).to(device)
        outputs[:, 0, 3] = 1 # SOS

        if use_teacher_forcing and y is not None:
            for di in range(1, target_length):
                out, hidden = self.decoder(out.to(dtype=torch.long), hidden, enc_out)
                outputs[:, di:di + 1] = out

                out = y[:, di].unsqueeze(1)
        else:
            for di in range(1, target_length):
                out, hidden = self.decoder(out.to(dtype=torch.long), hidden, enc_out)
                outputs[:, di:di + 1] = out

                out = out.argmax(dim=-1)
        return outputs


def generate_square_subsequent_mask(sz, cuda=True):
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)
    if cuda:
        return mask.cuda()
    return mask.cpu()

In [14]:
def decode_ctc(sent):
    if not sent:
        return sent
    last_token = sent[0]
    real_sent = []
    for token in sent[1:]:
        if last_token != '<CTC>' and token != last_token:
            real_sent.append(last_token)
        last_token = token
    return real_sent

In [15]:
class Trainer:
    def __init__(self, model: nn.Module, train_loader, val_loader, tf=0.25, lr=3e-4, betas=(0.9, 0.999),
                 project="ctc_translation", name='ctc_model', save_every=None, save_path='./'):
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.model = model
        self.save_path = save_path
        self.save_every = save_every
        self.name = name
        self.project = project
        self.tf = tf
        wandb.init(project=project, name=name)
        

    def train_epoch(self, cuda=True, clip=1):
        pass

    def test_epoch(self, cuda=True):
        pass

    def output(self, cuda=True):
        pass

    @staticmethod
    def log(epoch, train_loss, test_loss):
        wandb.log({
            'train': {
                'loss': train_loss
            },
            'val': {
                'loss': test_loss
            },
            'epoch': epoch
        })

    @staticmethod
    def log_train(train_loss):
        wandb.log({
            'train': {
                'loss': train_loss
            }
        })

    @staticmethod
    def log_test(test_loss):
        wandb.log({
            'test': {
                'loss': test_loss
            }
        })

    def checkpoint(self, epoch):
        torch.save(self.model.state_dict(), os.path.join(self.save_path, self.name + str(epoch) + '.ckpt'))

    def fit(self, max_epochs: int = 11, cuda=True, clip=1, log=False):
        for epoch in range(max_epochs):
            if self.save_every and epoch % self.save_every == 0:
                self.checkpoint(epoch)
            print('\rEpoch: %d' % epoch)
            self.output(cuda=cuda)
            train_loss = self.train_epoch(cuda=cuda, clip=clip)
            test_loss = self.test_epoch(cuda=cuda)
            if log:
                self.log(epoch, train_loss, test_loss)

        if self.save_every:
            self.checkpoint(max_epochs)

In [16]:
class TrLSTMTCTCrainer(Trainer):
    def __init__(self, model: nn.Module, train_loader, val_loader, tf=0.25, lr=3e-4, betas=(0.9, 0.999),
                 project="ctc_translation", name='ctc_model', save_every=None, save_path='./'):
        super().__init__(model, train_loader, val_loader, tf, lr, betas, project, name, save_every, save_path)
        self.ctc_criterion = nn.CTCLoss(blank=vocabs[0].t2i['<CTC>'], zero_infinity=True)

    def train_epoch(self, cuda=True, clip=1):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        self.model.train()
        total_loss = 0
        total_ctc_loss = 0
        for batch_idx, (tokens, targets, _) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            if cuda:
                tokens = tokens.cuda()
                targets = targets.cuda()
            # input_lengths = torch.full(size=(tokens.shape[0],), 
            #                 fill_value=tokens.shape[1], 
            #                 dtype=torch.long)
            target_lengths = (targets != 0).sum(dim=1)
            ctc = self.model(tokens.to(dtype=torch.long), targets.to(dtype=torch.long))
            input_lengths = (tokens != 0).sum(dim=1)
            loss = 0
            ctc_loss = self.ctc_criterion(ctc.permute(1, 0, 2).to(dtype=torch.float), targets.to(dtype=torch.long), 
                                          input_lengths=input_lengths, target_lengths=target_lengths)
            total_ctc_loss += ctc_loss.item()
            loss += ctc_loss
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), clip)
            self.optimizer.step()

            total_loss += loss.item()

            self.log_train(total_loss / (batch_idx + 1))
            print('\rTrain loss: %4f, Batch: %d of %d' % (
                total_loss / (batch_idx + 1), batch_idx + 1, len(self.train_loader)), end='')
        print()
        loss = total_loss / len(self.train_loader)
        ctc_loss = total_ctc_loss / len(self.train_loader)
        return loss

    def test_epoch(self, cuda=True):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        with torch.no_grad():
            self.model.eval()
            total_loss = 0
            total_ctc_loss = 0
            for batch_idx, (tokens, targets, _) in enumerate(self.val_loader):
                if cuda:
                    tokens = tokens.cuda()
                    targets = targets.cuda()
                # input_lengths = torch.full(size=(tokens.shape[0],), 
                #                 fill_value=tokens.shape[1], 
                #                 dtype=torch.long)
                target_lengths = (targets != 0).sum(dim=1)
                ctc = self.model(tokens.to(dtype=torch.long), targets.to(dtype=torch.long))
                input_lengths = (tokens != 0).sum(dim=1)

                loss = 0 
                ctc_loss = self.ctc_criterion(ctc.permute(1, 0, 2).to(dtype=torch.float), targets.to(dtype=torch.long), 
                                              input_lengths=input_lengths, target_lengths=target_lengths)
                total_ctc_loss += ctc_loss.item()
                loss += ctc_loss
                total_loss += loss.item()

                self.log_test(total_loss / (batch_idx + 1))
                print('\rVal loss: %4f, Batch: %d of %d' % (
                    total_loss / (batch_idx + 1), batch_idx + 1, len(self.val_loader)), end='')
            print()
            loss = total_loss / len(self.val_loader)
            ctc_loss = total_ctc_loss / len(self.val_loader)
            return loss

    def output(self, cuda=True):
        self.model.eval()
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        tokens, targets, _ = next(iter(self.val_loader))
        tokens = tokens[1:2].to(dtype=torch.long)
        targets = targets[1:2].to(dtype=torch.long)
        batch_size = tokens.shape[0]
        if cuda:
            tokens = tokens.cuda()
            targets = targets.cuda()
        ctc = self.model(tokens, targets)
        ctc = ctc.argmax(dim=-1)
        summ = '<SOS>'
        ctc_sent = []
        for di in range(1, targets.shape[1]):
            summ += vocabs[1].i2t[targets[0, di].cpu().detach().squeeze().item()] + ' '
        for di in range(ctc.shape[1]):
            ctc_sent.append(vocabs[1].i2t[ctc[0, di].cpu().detach().squeeze().item()])

        print(summ[:-1])
        print(' '.join(decode_ctc(ctc_sent)))

In [17]:
class TrTransformerCTCrainer(Trainer):
    def __init__(self, model: nn.Module, train_loader, val_loader, tf=0.25, lr=3e-4, betas=(0.9, 0.999),
                 project="ctc_translation", name='ctc_model', save_every=None, save_path='./'):
        super().__init__(model, train_loader, val_loader, tf, lr, betas, project, name, save_every, save_path)
        self.ctc_criterion = nn.CTCLoss(blank=vocabs[0].t2i['<CTC>'], zero_infinity=True)

    def train_epoch(self, cuda=True, clip=1):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        self.model.train()
        total_loss = 0
        total_ctc_loss = 0
        for batch_idx, (tokens, targets, _) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            if cuda:
                tokens = tokens.cuda()
                targets = targets.cuda()
            # input_lengths = torch.full(size=(tokens.shape[0],), 
            #                            fill_value=tokens.shape[1], 
            #                            dtype=torch.long)
            target_lengths = (targets != 0).sum(dim=1)
            ctc = self.model(tokens.to(dtype=torch.long).permute(1, 0), 
                             generate_square_subsequent_mask(tokens.shape[1], cuda))
            input_lengths = (tokens != 0).sum(dim=1)
            loss = 0
            ctc_loss = self.ctc_criterion(ctc.permute(1, 0, 2).to(dtype=torch.float), targets.to(dtype=torch.long), 
                                          input_lengths=input_lengths, target_lengths=target_lengths)
            total_ctc_loss += ctc_loss.item()
            loss += ctc_loss
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), clip)
            self.optimizer.step()

            total_loss += loss.item()

            self.log_train(total_loss / (batch_idx + 1))
            print('\rTrain loss: %4f, Batch: %d of %d' % (
                total_loss / (batch_idx + 1), batch_idx + 1, len(self.train_loader)), end='')
        print()
        loss = total_loss / len(self.train_loader)
        ctc_loss = total_ctc_loss / len(self.train_loader)
        return loss

    def test_epoch(self, cuda=True):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        with torch.no_grad():
            self.model.eval()
            total_loss = 0
            total_ctc_loss = 0
            for batch_idx, (tokens, targets, _) in enumerate(self.val_loader):
                if cuda:
                    tokens = tokens.cuda()
                    targets = targets.cuda()
                # input_lengths = torch.full(size=(tokens.shape[0],), 
                #                            fill_value=tokens.shape[1], 
                #                            dtype=torch.long)
                target_lengths = (targets != 0).sum(dim=1)
                ctc = self.model(tokens.to(dtype=torch.long).permute(1, 0), 
                                 generate_square_subsequent_mask(tokens.shape[1], cuda))
                input_lengths = (tokens != 0).sum(dim=1)

                loss = 0 
                ctc_loss = self.ctc_criterion(ctc.permute(1, 0, 2).to(dtype=torch.float), targets.to(dtype=torch.long), 
                                              input_lengths=input_lengths, target_lengths=target_lengths)
                total_ctc_loss += ctc_loss.item()
                loss += ctc_loss
                total_loss += loss.item()
            
                self.log_test(total_loss / (batch_idx + 1))
                print('\rVal loss: %4f, Batch: %d of %d' % (
                    total_loss / (batch_idx + 1), batch_idx + 1, len(self.val_loader)), end='')
            print()
            loss = total_loss / len(self.val_loader)
            ctc_loss = total_ctc_loss / len(self.val_loader)
            return loss

    def output(self, cuda=True):
        self.model.eval()
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        tokens, targets, _ = next(iter(self.val_loader))
        tokens = tokens[1:2].to(dtype=torch.long)
        targets = targets[1:2].to(dtype=torch.long)
        batch_size = tokens.shape[0]
        if cuda:
            tokens = tokens.cuda()
            targets = targets.cuda()
        ctc = self.model(tokens.permute(1, 0), 
                         generate_square_subsequent_mask(tokens.shape[1], cuda))
        ctc = ctc.argmax(dim=-1)
        summ = '<SOS>'
        ctc_sent = []
        for di in range(1, targets.shape[1]):
            summ += vocabs[1].i2t[targets[0, di].cpu().detach().squeeze().item()] + ' '
        for di in range(ctc.shape[1]):
            ctc_sent.append(vocabs[1].i2t[ctc[0, di].cpu().detach().squeeze().item()])

        print(summ[:-1])
        print(' '.join(decode_ctc(ctc_sent)))

In [18]:
def decompose_alignments(alignments, cuda=True):
    groups = torch.cumsum(alignments.sum(2), 1).unsqueeze(2)
    lengths = torch.cumsum(torch.clamp(alignments.sum(1), 0, 3), 1).unsqueeze(1)

    grouping_matrix = torch.zeros((alignments.shape[0], alignments.shape[1], alignments.shape[1] * 3)) + torch.arange(1, 1 + alignments.shape[1] * 3).unsqueeze(0).unsqueeze(1)
    if cuda:
        grouping_matrix = grouping_matrix.cuda()
    grouping_matrix[:, 0, :] = torch.clamp(groups[:, 0, :] - grouping_matrix[:, 0, :] + 1, 0, 1)
    grouping_matrix[:, 1:, :] = torch.clamp(groups[:, 1:, :] - grouping_matrix[:, 1:, :] + 1, 0, 1) * torch.clamp(grouping_matrix[:, 1:, :] - groups[:, :-1, :], 0, 1)

    duplication_matrix = torch.zeros((alignments.shape[0], alignments.shape[1] * 3, alignments.shape[2])) + torch.arange(1, 1 + alignments.shape[1] * 3).unsqueeze(0).unsqueeze(2)
    if cuda:
        duplication_matrix = duplication_matrix.cuda()
    duplication_matrix[:, :, 0] = torch.clamp(lengths[:, :, 0] - duplication_matrix[:, :, 0] + 1, 0, 1)
    duplication_matrix[:, :, 1:] = torch.clamp(lengths[:, :, 1:] - duplication_matrix[:, :, 1:] + 1, 0, 1) * torch.clamp(duplication_matrix[:, :, 1:] - lengths[:, :, :-1], 0, 1)
    
    permutation_matrix = (grouping_matrix.permute(0, 2, 1) / (alignments.sum(2).unsqueeze(1) + 1e-16))@alignments@(duplication_matrix.permute(0, 2, 1) / (alignments.sum(1).unsqueeze(2) + 1e-16))
    eye = torch.eye(permutation_matrix.shape[1], permutation_matrix.shape[2])
    if cuda:
        eye = eye.cuda()

    for i in range(permutation_matrix.shape[0]):
        x = int(permutation_matrix[i].sum())
        permutation_matrix[i][x:, x:] = eye[x:, x:]

    return duplication_matrix, permutation_matrix, grouping_matrix

In [19]:
class AligNARTrainer(Trainer):
    def __init__(self, model: nn.Module, train_loader, val_loader, tf=0.25, lr=3e-4, betas=(0.9, 0.999),
                 project="ctc_translation", name='ctc_model', save_every=None, save_path='./', lmbda=0.5):
        super().__init__(model, train_loader, val_loader, tf, lr, betas, project, name, save_every, save_path)
        self.lmbda = lmbda
        self.groups_loss = GroupsLoss()
        self.permutation_loss = nn.KLDivLoss(reduction="mean")
        self.cross_enthropy = nn.CrossEntropyLoss()

    def train_epoch(self, cuda=True, clip=1):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        self.model.train()
        total_grouping_loss = 0
        total_permutation_loss = 0
        total_duplication_loss = 0
        total_ce_loss = 0
        total_loss = 0
        for batch_idx, (tokens, targets, alignments) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            if cuda:
                tokens = tokens.cuda()
                targets = targets.cuda()
                alignments = alignments.cuda()

            duplication_matrix, permutation_matrix, grouping_matrix = decompose_alignments(alignments, cuda=cuda)
            
            output, (duplication_probs, duplication_matrix_p, permutation_matrix_p, grouping_probs, grouping_matrix_p) = self.model(tokens.to(dtype=torch.long).permute(1, 0), 
                                                                                                                                    generate_square_subsequent_mask(tokens.shape[1], cuda),
                                                                                                                                    [duplication_matrix.to(dtype=torch.float),
                                                                                                                                     permutation_matrix.to(dtype=torch.float), 
                                                                                                                                     grouping_matrix.to(dtype=torch.float)])

            duplication_loss = self.groups_loss(duplication_probs).mean()
            permutation_loss = self.permutation_loss(torch.nn.functional.log_softmax(permutation_matrix_p, dim=-1), 
                                                     permutation_matrix).mean()
            grouping_loss = self.groups_loss(grouping_probs).mean()
            ce_loss = self.cross_enthropy(output.permute(0, 2, 1), targets).mean()


            loss = ce_loss + self.lmbda * (duplication_loss + permutation_loss + grouping_loss)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), clip)
            self.optimizer.step()

            total_loss += loss.item()
            total_grouping_loss += grouping_loss.item()
            total_permutation_loss += permutation_loss.item()
            total_duplication_loss += duplication_loss.item()
            total_ce_loss += ce_loss.item()

            self.log_train(total_loss / (batch_idx + 1), 
                           total_duplication_loss / (batch_idx + 1), 
                           total_permutation_loss / (batch_idx + 1),
                           total_grouping_loss / (batch_idx + 1),
                           total_ce_loss / (batch_idx + 1))
            print('\rTrain loss: %4f, Batch: %d of %d' % (
                total_loss / (batch_idx + 1), batch_idx + 1, len(self.train_loader)), end='')
        print()
        loss = [total_loss / len(self.train_loader), 
                total_duplication_loss / len(self.train_loader), 
                total_permutation_loss / len(self.train_loader), 
                total_grouping_loss / len(self.train_loader), 
                total_ce_loss / len(self.train_loader)]
        return loss

    def test_epoch(self, cuda=True):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        with torch.no_grad():
            self.model.eval()
            total_grouping_loss = 0
            total_permutation_loss = 0
            total_duplication_loss = 0
            total_ce_loss = 0
            total_loss = 0
            for batch_idx, (tokens, targets, alignments) in enumerate(self.val_loader):
                if cuda:
                    tokens = tokens.cuda()
                    targets = targets.cuda()
                    alignments = alignments.cuda()

                targets = targets.repeat(1, 3)
                targets[:, targets.shape[1] // 3:] = 0

                duplication_matrix, permutation_matrix, grouping_matrix = decompose_alignments(alignments, cuda=cuda)

                output, (duplication_probs, duplication_matrix_p, permutation_matrix_p, grouping_probs, grouping_matrix_p) = self.model(tokens.to(dtype=torch.long).permute(1, 0), 
                                                                                                                                        generate_square_subsequent_mask(tokens.shape[1], cuda))

                duplication_loss = self.groups_loss(duplication_probs).mean()
                permutation_loss = self.permutation_loss(torch.nn.functional.log_softmax(permutation_matrix_p, dim=-1), 
                                                         permutation_matrix).mean()
                grouping_loss = self.groups_loss(grouping_probs).mean()
                ce_loss = self.cross_enthropy(output.permute(0, 2, 1), targets).mean()
                
                loss = ce_loss + self.lmbda * (duplication_loss + permutation_loss + grouping_loss)
                total_loss += loss.item()
                total_grouping_loss += grouping_loss.item()
                total_permutation_loss += permutation_loss.item()
                total_duplication_loss += duplication_loss.item()
                total_ce_loss += ce_loss.item()

                self.log_test(total_loss / (batch_idx + 1), 
                              total_duplication_loss / (batch_idx + 1), 
                              total_permutation_loss / (batch_idx + 1),
                              total_grouping_loss / (batch_idx + 1),
                              total_ce_loss / (batch_idx + 1))
                print('\rVal loss: %4f, Batch: %d of %d' % (
                    total_loss / (batch_idx + 1), batch_idx + 1, len(self.val_loader)), end='')
            print()
            loss = [total_loss / len(self.val_loader), 
                    total_duplication_loss / len(self.val_loader), 
                    total_permutation_loss / len(self.val_loader), 
                    total_grouping_loss / len(self.val_loader), 
                    total_ce_loss / len(self.val_loader)]
            return loss

    def output(self, cuda=True):
        self.model.eval()
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        tokens, targets, alignments = next(iter(self.val_loader))
        tokens = tokens[1:2].to(dtype=torch.long)
        targets = targets[1:2].to(dtype=torch.long)
        batch_size = tokens.shape[0]
        if cuda:
            tokens = tokens.cuda()
            targets = targets.cuda()
        output, _ = self.model(tokens.to(dtype=torch.long).permute(1, 0),
                               generate_square_subsequent_mask(tokens.shape[1], cuda))
        output = output.argmax(dim=-1)
        
        summ = '<SOS>'
        sent = []
        for di in range(1, targets.shape[1]):
            summ += vocabs[1].i2t[targets[0, di].cpu().detach().squeeze().item()] + ' '
        for di in range(output.shape[1]):
            sent.append(vocabs[1].i2t[output[0, di].cpu().detach().squeeze().item()])

        print(summ[:-1])
        print(' '.join(sent))


    @staticmethod
    def log(epoch, train_loss, test_loss):
        wandb.log({
            'train': {
                'loss': train_loss[0],
                'duplication_loss': train_loss[1],
                'permutation_loss': train_loss[2],
                'grouping_loss': train_loss[3],
                'cross_enthropy_loss': train_loss[4]
            },
            'val': {
                'loss': test_loss[0],
                'duplication_loss': test_loss[1],
                'permutation_loss': test_loss[2],
                'grouping_loss': test_loss[3],
                'cross_enthropy_loss': test_loss[4]
            },
            'epoch': epoch
        })

    @staticmethod
    def log_train(train_loss, duplication_loss, permutation_loss, grouping_loss, ce_loss):
        wandb.log({
            'train': {
                'loss': train_loss,
                'duplication_loss': duplication_loss,
                'permutation_loss': permutation_loss,
                'grouping_loss': grouping_loss,
                'cross_enthropy_loss': ce_loss
            }
        })

    @staticmethod
    def log_test(test_loss, duplication_loss, permutation_loss, grouping_loss, ce_loss):
        wandb.log({
            'test': {
                'loss': test_loss,
                'duplication_loss': duplication_loss,
                'permutation_loss': permutation_loss,
                'grouping_loss': grouping_loss,
                'cross_enthropy_loss': ce_loss
            }
        })

    
    

In [20]:
class CTCAligNARTrainer(Trainer):
    def __init__(self, model: nn.Module, train_loader, val_loader, tf=0.25, lr=3e-4, betas=(0.9, 0.999),
                 project="ctc_translation", name='ctc_model', save_every=None, save_path='./', lmbda=0.5):
        super().__init__(model, train_loader, val_loader, tf, lr, betas, project, name, save_every, save_path)
        self.lmbda = lmbda
        self.groups_loss = GroupsLoss()
        self.permutation_loss = nn.KLDivLoss(reduction="mean")
        self.ctc_criterion = nn.CTCLoss(blank=vocabs[0].t2i['<CTC>'], zero_infinity=True)

    def train_epoch(self, cuda=True, clip=1):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        self.model.train()
        total_loss = 0
        total_permutation_loss = 0
        total_ctc_loss = 0
        for batch_idx, (tokens, targets, alignments) in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            if cuda:
                tokens = tokens.cuda()
                targets = targets.cuda()
                alignments = alignments.cuda()
            # input_lengths = torch.full(size=(tokens.shape[0],), 
            #                            fill_value=tokens.shape[1], 
            #                            dtype=torch.long)
            target_lengths = (targets != 0).sum(dim=1)

            duplication_matrix, permutation_matrix, grouping_matrix = decompose_alignments(alignments, cuda=cuda)


            ctc, (duplication_matrix_p, permutation_matrix_p) = self.model(tokens.to(dtype=torch.long).permute(1, 0), 
                                                                       generate_square_subsequent_mask(tokens.shape[1], cuda),
                                                                        [duplication_matrix.to(dtype=torch.float), permutation_matrix.to(dtype=torch.float)])
            input_lengths = (tokens != 0).sum(dim=1)
            loss = 0
            ctc_loss = self.ctc_criterion(ctc.permute(1, 0, 2).to(dtype=torch.float), targets.to(dtype=torch.long), 
                                          input_lengths=input_lengths, target_lengths=target_lengths)
            permutation_loss = self.permutation_loss(torch.nn.functional.log_softmax(permutation_matrix_p, dim=-1),
                                                     permutation_matrix).mean()
            loss += ctc_loss + permutation_loss
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), clip)
            self.optimizer.step()

            total_loss += loss.item()
            total_ctc_loss += ctc_loss.item()
            total_permutation_loss += permutation_loss.item()

            self.log_train(total_loss / (batch_idx + 1), 
                           total_permutation_loss / (batch_idx + 1), 
                           total_ctc_loss / (batch_idx + 1))
            print('\rTrain loss: %4f, Batch: %d of %d' % (
                total_loss / (batch_idx + 1), batch_idx + 1, len(self.train_loader)), end='')
        print()
        loss = [total_loss / len(self.train_loader),
                total_permutation_loss / len(self.train_loader),
                total_ctc_loss / len(self.train_loader)]
        return loss

    def test_epoch(self, cuda=True):
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        with torch.no_grad():
            self.model.eval()
            total_loss = 0
            total_permutation_loss = 0
            total_ctc_loss = 0
            for batch_idx, (tokens, targets, alignments) in enumerate(self.val_loader):
                if cuda:
                    tokens = tokens.cuda()
                    targets = targets.cuda()
                    alignments = alignments.cuda()
                # input_lengths = torch.full(size=(tokens.shape[0],), 
                #                            fill_value=tokens.shape[1], 
                #                            dtype=torch.long)
                target_lengths = (targets != 0).sum(dim=1)
                targets = targets.repeat(1, 3)
                targets[:, targets.shape[1] // 3:] = 0

                duplication_matrix, permutation_matrix, grouping_matrix = decompose_alignments(alignments, cuda=cuda)

                ctc, (duplication_matrix_p, permutation_matrix_p) = self.model(tokens.to(dtype=torch.long).permute(1, 0), 
                                                                               generate_square_subsequent_mask(tokens.shape[1], cuda))
                input_lengths = (tokens != 0).sum(dim=1)

                loss = 0 
                ctc_loss = self.ctc_criterion(ctc.permute(1, 0, 2).to(dtype=torch.float), targets.to(dtype=torch.long), 
                                              input_lengths=input_lengths, target_lengths=target_lengths)
                permutation_loss = self.permutation_loss(torch.nn.functional.log_softmax(permutation_matrix_p, dim=-1),
                                                         permutation_matrix).mean()
                loss += ctc_loss + permutation_loss

                total_loss += loss.item()
                total_ctc_loss += ctc_loss.item()
                total_permutation_loss += permutation_loss.item()
            
                self.log_test(total_loss / (batch_idx + 1), 
                              total_permutation_loss / (batch_idx + 1), 
                              total_ctc_loss / (batch_idx + 1))
                print('\rVal loss: %4f, Batch: %d of %d' % (
                    total_loss / (batch_idx + 1), batch_idx + 1, len(self.val_loader)), end='')
            print()
            loss = [total_loss / len(self.val_loader),
                    total_permutation_loss / len(self.val_loader),
                    total_ctc_loss / len(self.val_loader)]
            return loss

    def output(self, cuda=True):
        self.model.eval()
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        tokens, targets, _ = next(iter(self.val_loader))
        tokens = tokens[1:2].to(dtype=torch.long)
        targets = targets[1:2].to(dtype=torch.long)
        batch_size = tokens.shape[0]
        if cuda:
            tokens = tokens.cuda()
            targets = targets.cuda()
        ctc, _ = self.model(tokens.permute(1, 0), 
                         generate_square_subsequent_mask(tokens.shape[1], cuda))
        ctc = ctc.argmax(dim=-1)
        summ = '<SOS>'
        ctc_sent = []
        for di in range(1, targets.shape[1]):
            summ += vocabs[1].i2t[targets[0, di].cpu().detach().squeeze().item()] + ' '
        for di in range(ctc.shape[1]):
            ctc_sent.append(vocabs[1].i2t[ctc[0, di].cpu().detach().squeeze().item()])

        print(summ[:-1])
        print(' '.join(decode_ctc(ctc_sent)))
    
    
    @staticmethod
    def log(epoch, train_loss, test_loss):
        wandb.log({
            'train': {
                'loss': train_loss[0],
                'permutation_loss': train_loss[1],
                'ctc_loss': train_loss[2]
            },
            'val': {
                'loss': test_loss[0],
                'permutation_loss': test_loss[1],
                'ctc_loss': test_loss[2]
            },
            'epoch': epoch
        })

    @staticmethod
    def log_train(train_loss, permutation_loss, ctc_loss):
        wandb.log({
            'train': {
                'loss': train_loss,
                'permutation_loss': permutation_loss,
                'ctc_loss': ctc_loss
            }
        })

    @staticmethod
    def log_test(test_loss, permutation_loss, ctc_loss):
        wandb.log({
            'test': {
                'loss': test_loss,
                'permutation_loss': permutation_loss,
                'ctc_loss': ctc_loss
            }
        })

In [21]:
class TrBiLSTMTrainer(Trainer):
     def __init__(self, model: nn.Module, train_loader, val_loader, tf=0.5, lr=3e-4, betas=(0.9, 0.999),
                  project="ctc_translation", name='ctc_model', save_every=None, save_path='./',):
         super().__init__(model, train_loader, val_loader, tf, lr, betas, project, name, save_every, save_path)
         self.criterion = nn.CrossEntropyLoss()

     def train_epoch(self, cuda=True, clip=1):
         if cuda:
             self.model.cuda()
         else:
             self.model.cpu()
         self.model.train()
         total_loss = 0

         for batch_idx, (tokens, targets, _) in enumerate(self.train_loader):
             self.optimizer.zero_grad()
             if cuda:
                 tokens = tokens.cuda()
                 targets = targets.cuda()
             use_teacher_forcing = True if random.random() < self.tf else False
             outputs = self.model(tokens.to(dtype=torch.long), 
                                  targets.to(dtype=torch.long), use_teacher_forcing)

             loss = self.criterion(outputs[:, 1:].reshape(-1, self.model.decoder_index_dim),
                                   targets[:, 1:].flatten())
             loss.backward()
             nn.utils.clip_grad_norm_(self.model.parameters(), clip)
             self.optimizer.step()

             total_loss += loss.item()

             self.log_train(total_loss / (batch_idx + 1))
             print('\rTrain loss: %4f, Batch: %d of %d' % (total_loss / (batch_idx + 1), 
                                                           batch_idx + 1, len(self.train_loader)), end='')
         print()
         loss = total_loss / len(self.train_loader)
         return loss

     def test_epoch(self, cuda=True):
         if cuda:
             self.model.cuda()
         else:
             self.model.cpu()
         with torch.no_grad():
             self.model.eval()
             total_loss = 0
             for batch_idx, (tokens, targets, _) in enumerate(self.val_loader):
                 if cuda:
                     tokens = tokens.cuda()
                     targets = targets.cuda()
                 use_teacher_forcing = False
                 outputs = self.model(tokens.to(dtype=torch.long), 
                                      targets.to(dtype=torch.long),
                                      use_teacher_forcing)

                 loss = self.criterion(outputs[:, 1:].reshape(-1, self.model.decoder_index_dim),
                                       targets[:, 1:].flatten())
                 total_loss += loss.item()

                 self.log_test(total_loss / (batch_idx + 1))
                 print('\rVal loss: %4f, Batch: %d of %d' % (total_loss / (batch_idx + 1), 
                                                             batch_idx + 1, len(self.val_loader)), end='')
             print()
             loss = total_loss / len(self.val_loader)
             return loss

     def output(self, cuda=True):
        self.model.eval()
        if cuda:
            self.model.cuda()
        else:
            self.model.cpu()
        tokens, targets, _ = next(iter(self.val_loader))
        tokens = tokens[1:2].to(dtype=torch.long)
        targets = targets[1:2].to(dtype=torch.long)
        batch_size = tokens.shape[0]
        if cuda:
            tokens = tokens.cuda()
            targets = targets.cuda()
        use_teacher_forcing = False
        output = self.model(tokens, None, False)
        output = output.argmax(dim=-1)
        
        summ = '<SOS>'
        sent = []
        for di in range(1, targets.shape[1]):
            summ += vocabs[1].i2t[targets[0, di].cpu().detach().squeeze().item()] + ' '
        for di in range(output.shape[1]):
            sent.append(vocabs[1].i2t[output[0, di].cpu().detach().squeeze().item()])

        print(summ[:-1])
        print(' '.join(sent))

In [22]:
import gc
gc.collect()

72

In [ ]:
bilstm_model = Seq2Seq(len(vocabs[0].t2i), len(vocabs[1].t2i))
# ctc_only_model.load_state_dict(torch.load('/content/drive/MyDrive/CTC_in_NLP/checkpoints/bilstm_ctc_only_25610.ckpt'))
bilstm_trainer = TrBiLSTMTrainer(bilstm_model, train_loader, val_loader, 
                                  lr=3e-4, save_every=1, name='bilstm_de_en', 
                                  save_path=f'/content/drive/MyDrive/CTC_in_NLP/checkpoints')
bilstm_trainer.fit(log=True, max_epochs=3)

In [28]:
alignart_model = AligNART(len(vocabs[0].t2i), len(vocabs[1].t2i))
alignart_model.load_state_dict(torch.load('/content/drive/MyDrive/CTC_in_NLP/checkpoints/alignart_256_de_en1.ckpt'))
# alignart_trainer = AligNARTrainer(alignart_model, train_loader, val_loader, 
#                                   lr=3e-4, save_every=1, name='alignart_de_en', 
#                                   save_path=f'/content/drive/MyDrive/CTC_in_NLP/checkpoints')
# alignart_trainer.fit(log=True, max_epochs=5)

<All keys matched successfully>

In [29]:
ctc_aligner_model = CTCAligner(len(vocabs[0].t2i), len(vocabs[1].t2i))
ctc_aligner_model.load_state_dict(torch.load('/content/drive/MyDrive/CTC_in_NLP/checkpoints/ctc_aligner_256_de_en1.ckpt'))
# ctc_aligner_trainer = CTCAligNARTrainer(ctc_aligner_model, train_loader, val_loader, 
#                                   lr=3e-4, save_every=1, name='ctc_aligner_256_de_en', 
#                                   save_path=f'/content/drive/MyDrive/CTC_in_NLP/checkpoints')
# ctc_aligner_trainer.fit(log=True, max_epochs=5)

<All keys matched successfully>

In [30]:
ctc_only_model = Seq2CTC(len(vocabs[0].t2i), len(vocabs[1].t2i))
ctc_only_model.load_state_dict(torch.load('/content/drive/MyDrive/CTC_in_NLP/checkpoints/bilstm_ctc_only_256_de_en4.ckpt'))
# ctc_only_trainer = TrLSTMTCTCrainer(ctc_only_model, train_loader, val_loader, 
#                                   lr=3e-4, save_every=1, name='bilstm_ctc_only_256_de_en', 
#                                   save_path=f'/content/drive/MyDrive/CTC_in_NLP/checkpoints')
# ctc_only_trainer.fit(log=True, max_epochs=5) 

<All keys matched successfully>

In [34]:
ctc_only_transformer_model = Transformer2CTC(len(vocabs[0].t2i), len(vocabs[1].t2i))
# encoder_index_dim, decoder_index_dim, heads, hidden_dim, layers
ctc_only_transformer_model.load_state_dict(torch.load('/content/drive/MyDrive/CTC_in_NLP/checkpoints/transformer_ctc_only_de_en4.ckpt'))
# ctc_only_trainer = TrTransformerCTCrainer(ctc_only_transformer_model, train_loader, val_loader, 
#                                   lr=3e-4, save_every=1, name='transformer_ctc_only_de_en', 
#                                   save_path=f'/content/drive/MyDrive/CTC_in_NLP/checkpoints')
# ctc_only_trainer.fit(log=True, max_epochs=5)

<All keys matched successfully>

In [36]:
ctc_only_t2t_model = Transformer2TransformerCTC(len(vocabs[0].t2i), len(vocabs[1].t2i))
# encoder_index_dim, decoder_index_dim, heads, hidden_dim, layers
ctc_only_t2t_model.load_state_dict(torch.load('/content/drive/MyDrive/CTC_in_NLP/checkpoints/transformer2transformer_ctc_only_de_en5.ckpt'))
# ctc_only_trainer = TrTransformerCTCrainer(ctc_only_t2t_model, train_loader, val_loader, 
#                                   lr=3e-4, save_every=1, name='transformer2transformer_ctc_only_de_en', 
#                                   save_path=f'/content/drive/MyDrive/CTC_in_NLP/checkpoints')
# ctc_only_trainer.fit(log=True, max_epochs=5)

<All keys matched successfully>

In [75]:
def cut_on_eos(tokens_iter):
    for token in tokens_iter:
        if token == '<EOS>':
            break
        yield token

def remove_tech_tokens(tokens_iter, tokens_to_remove=['<SOS>', '<UNK>', '<PAD>']):
    return [x for x in tokens_iter if x not in tokens_to_remove]

def generate_translation(src, trg, model):
    model.eval()

    ctc = model(src, trg)
    output = ctc[:, 1:].argmax(-1)

    source = remove_tech_tokens(cut_on_eos([vocabs[0].i2t[x] for x in list(src[0, :].cpu().detach().numpy())]))
    original = remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(trg[0, :].cpu().detach().numpy())]))
    generated = decode_ctc(remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(output[0, :].cpu().detach().squeeze().numpy())])))
    
    print('Source: {}'.format(' '.join(source)))
    print('Original: {}'.format(' '.join(original)))
    print('Generated: {}'.format(' '.join(generated)))
    print()

def generate_trf_translation(src, trg, model):
    model.eval()

    ctc = model(src.permute(1, 0).cpu(), 
                generate_square_subsequent_mask(src.shape[1], False))
    output = ctc[:, 1:].argmax(-1)

    source = remove_tech_tokens(cut_on_eos([vocabs[0].i2t[x] for x in list(src[0, :].cpu().detach().numpy())]))
    original = remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(trg[0, :].cpu().detach().numpy())]))
    generated = decode_ctc(remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(output[0, :].cpu().detach().squeeze().numpy())])))
    
    print('Source: {}'.format(' '.join(source)))
    print('Original: {}'.format(' '.join(original)))
    print('Generated: {}'.format(' '.join(generated)))
    print()

def generate_trf_align_translation(src, trg, model):
    model.eval()

    ctc, _ = model(src.permute(1, 0).cpu(), 
                generate_square_subsequent_mask(src.shape[1], False))
    output = ctc[:, 1:].argmax(-1)

    source = remove_tech_tokens(cut_on_eos([vocabs[0].i2t[x] for x in list(src[0, :].cpu().detach().numpy())]))
    original = remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(trg[0, :].cpu().detach().numpy())]))
    generated = decode_ctc(remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(output[0, :].cpu().detach().squeeze().numpy())])))
    
    print('Source: {}'.format(' '.join(source)))
    print('Original: {}'.format(' '.join(original)))
    print('Generated: {}'.format(' '.join(generated)))
    print()

def generate_trf_alignart_translation(src, trg, model):
    model.eval()

    ctc, _ = model(src.permute(1, 0).cpu(), 
                generate_square_subsequent_mask(src.shape[1], False))
    output = ctc[:, 1:].argmax(-1)

    source = remove_tech_tokens(cut_on_eos([vocabs[0].i2t[x] for x in list(src[0, :].cpu().detach().numpy())]))
    original = remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(trg[0, :].cpu().detach().numpy())]))
    generated = remove_tech_tokens(cut_on_eos([vocabs[1].i2t[x] for x in list(output[0, :].cpu().detach().squeeze().numpy())]))
    
    print('Source: {}'.format(' '.join(source)))
    print('Original: {}'.format(' '.join(original)))
    print('Generated: {}'.format(' '.join(generated)))
    print()

def get_text(x):
     generated = remove_tech_tokens(cut_on_eos([vocabs[1].i2t[elem] for elem in list(x)]))
     return generated


def calculate_ctc_bleu(model, loader):
    model.eval()
    generated = []
    original = []
    for src, trg, _ in loader:
        ctc = model(src.cuda(), trg.cuda())
        output = ctc[:, 1:].argmax(-1)

        original.extend([decode_ctc(get_text(x)) for x in trg.cpu().numpy()])
        generated.extend([decode_ctc(get_text(x)) for x in output.detach().cpu().squeeze().numpy()])
    
    return corpus_bleu([[text] for text in original], generated) * 100


def calculate_aligner_bleu(model, loader):
    model.eval()
    generated = []
    original = []
    for src, trg, _ in loader:
        output, _ = model(src.permute(1, 0).cuda(), 
                         generate_square_subsequent_mask(src.shape[1], True))
        output = output[:, 1:].argmax(-1)

        original.extend([get_text(x) for x in trg.cpu().numpy()])
        generated.extend([get_text(x) for x in output.detach().cpu().squeeze().numpy()])
    
    return corpus_bleu([[text] for text in original], generated) * 100


def calculate_ctc_aligner_bleu(model, loader):
    model.eval()
    generated = []
    original = []
    for src, trg, _ in loader:
        output, _ = model(src.permute(1, 0).cuda(), 
                         generate_square_subsequent_mask(src.shape[1], True))
        output = output[:, 1:].argmax(-1)

        original.extend([decode_ctc(get_text(x)) for x in trg.cpu().numpy()])
        generated.extend([decode_ctc(get_text(x)) for x in output.detach().cpu().squeeze().numpy()])
    
    return corpus_bleu([[text] for text in original], generated) * 100


def calculate_ctc_trf_bleu(model, loader):
    model.eval()
    generated = []
    original = []
    for src, trg, _ in loader:
        ctc = model(src.permute(1, 0).cuda(), 
                    generate_square_subsequent_mask(src.shape[1], True))
        output = ctc[:, 1:].argmax(-1)

        original.extend([decode_ctc(get_text(x)) for x in trg.cpu().numpy()])
        generated.extend([decode_ctc(get_text(x)) for x in output.detach().cpu().squeeze().numpy()])
    
    return corpus_bleu([[text] for text in original], generated) * 100

In [67]:
%%time
calculate_ctc_bleu(ctc_only_model.cuda(), val_loader)

CPU times: user 1.88 s, sys: 27 ms, total: 1.9 s
Wall time: 2.45 s


20.20995532654378

In [68]:
%%time
calculate_ctc_trf_bleu(ctc_only_t2t_model.eval().cuda(), val_loader)

CPU times: user 939 ms, sys: 7.97 ms, total: 947 ms
Wall time: 1.01 s


18.35489341856907

In [72]:
%%time
calculate_ctc_trf_bleu(ctc_only_transformer_model.eval().cuda(), val_loader)

CPU times: user 1.05 s, sys: 1.89 ms, total: 1.06 s
Wall time: 1.07 s


21.90116777002967

In [73]:
%%time
calculate_aligner_bleu(alignart_model.eval().cuda(), val_loader)

CPU times: user 2.39 s, sys: 17 ms, total: 2.41 s
Wall time: 2.51 s


20.84921190003277

In [74]:
%%time
calculate_ctc_aligner_bleu(ctc_aligner_model.eval().cuda(), val_loader)

CPU times: user 2.28 s, sys: 21.9 ms, total: 2.3 s
Wall time: 2.32 s


23.00497336772189